<a href="https://colab.research.google.com/github/Gauravmehra1/Airbnb/blob/main/sql_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Given two tables below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company’s average salary.

In [ ]:
CREATE  TABLE  salary(
id integer,
employee_id INTEGER,
amount INTEGER,
pay_date TIMESTAMP
	)
	
insert into salary
values
(1,1,9000,' 2017-03-31'),
(2,2,6000,'2017-03-31'),
(3,2,10000,' 2017-03-31'),
(4,1,7000,'2017-02-28'),
(5,2,6000,' 2017-02-28'),
(6,2,8000,'2017-02-28')

SELECT * from salary 

CREATE table dept (
employee_id INTEGER,
department_id integer
)

insert into dept
values
(1,1),
(2,2),
(3,2)

select * from dept

CREATE TEMPORARY table t1 AS( 

   select to_char(pay_date,'yyyy-mm') as pay_month, department_id,
   avg (amount) over(PARTITION by date_part('month',pay_date), department_id) as dept_avg,
   avg (amount) over(PARTITION by date_part('month',pay_date)) as comp_avg
   from salary as s join dept as d
   using (employee_id)
)

select * from t1

select  distinct pay_month, department_id,
     case when dept_avg > comp_avg then 'higher'
	 when dept_avg = comp_avg then 'same'
	 else 'lower'
	 end as comprasion 
from t1
order by pay_month desc

2. Write an SQL query to report the students (student_id, student_name) being “quiet” in ALL exams. A “quiet” student is the one who took at least one exam and didn’t score neither the high score nor the low score.


In [ ]:
CREATE table student(
     student_Id serial primary key ,
     student_number varchar(100)
)

insert into student
values
(1, 'Daniel' ),
(2 , 'jade'),
(3 , 'stella'),
(4 , 'Jonathan'),
(5 , 'will')


select * from student 

create table exam (
   exam_id integer,
   student_id integer references student(student_id),
   score integer
)

insert into exam
values
(10,1,70),
(10,2,80),
(10,3,90),
(20,1,80),
(30,1,70),
(30,3,80),
(30,4,90),
(40,1,60),
(40,2,70),
(40,4,80)

select * from exam

create  TEMPORARY table t2 as (
  select student_id
	from(
	    select*,
	    min (score) over main_window as least,
	    max (score) over main_window as most
	from exam 
	window main_window as ( PARTITION by exam_id)
	)as a
 WHERE least = score or most = score
)	
  
  select * from t2
  
select distinct student_id,
            student_number
from exam join student
using(student_id)
where student_id != all(select student_id from t2)
order by student_id
